In [ ]:
from pyhive import hive
import psycopg2
import pandas as pd
import hashlib
import time
import math

class MobileConnection:
    def __init__(self, dbname, user, password, host, port):
        self.dbname = dbname
        self.user = user
        self.password = password
        self.host = host
        self.port = port
        self.connection = None
        self.cursor = None

    def connect(self):
        """Creates a connection to the Postgres Mobile database."""
        try:
            self.connection = psycopg2.connect(
                dbname=self.dbname,
                user=self.user,
                password=self.password,
                host=self.host,
                port=self.port
            )
            self.connection.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
            self.cursor = self.connection.cursor()
            return True
        except Exception as e:
            print(f"Error creating PostgreSQL Mobile connection: {e}")
            return False

    def close(self):
        """Closes the connection to the Mobile database."""
        if self.connection:
            try:
                self.connection.close()
            except Exception as e:
                print(f"Error closing connection: {e}")

    def execute_query(self, query):
        """Executes the given query and returns the results."""
        try:
            cursor = self.connection.cursor()
            cursor.execute(query)
            columns = [desc[0] for desc in cursor.description]
            results = cursor.fetchall()
            return results, columns
        except Exception as e:
            print(f"Error executing query: {e}")
            return None
        finally:
            cursor.close()


class PostgresConnection:
    def __init__(self, dbname, user, password, host, port):
        self.dbname = dbname
        self.user = user
        self.password = password
        self.host = host
        self.port = port
        self.connection = None
        self.cursor = None

    def connect(self):
        """Creates a connection to the PostgreSQL database."""
        try:
            self.connection = psycopg2.connect(
                dbname=self.dbname,
                user=self.user,
                password=self.password,
                host=self.host,
                port=self.port
            )
            self.connection.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
            self.cursor = self.connection.cursor()
            return True
        except Exception as e:
            print(f"Error creating PostgreSQL connection: {e}")
            return False

    def execute_query(self, query, params=None):
        """Executes a query with optional parameters for SELECT, INSERT, or UPDATE operations."""
        try:
            # Execute the query with parameters
            self.cursor.execute(query, params)

            # Check if the query is a SELECT statement
            if query.strip().upper().startswith("SELECT"):
                # Fetch and return results for SELECT queries
                return self.cursor.fetchall()
            else:
                # Commit transaction for non-SELECT queries (e.g., INSERT, UPDATE)
                self.connection.commit()
                return True  # Return True to indicate success for INSERT/UPDATE queries
        except Exception as e:
            print(f"Error executing PostgreSQL query: {e}")
            return None

    def close(self):
        """Closes the PostgreSQL database connection."""
        if self.connection:
            try:
                self.connection.close()
            except Exception as e:
                print(f"Error closing PostgreSQL connection: {e}")


class DataFetcher:
    def __init__(self, mob_conn, pg_conn, batch_size=50000, polling_interval=600):
        self.mob_conn = mob_conn
        self.pg_conn = pg_conn
        self.batch_size = batch_size
        self.polling_interval = polling_interval
        self.last_processed_value = '1900-01-01 00:00:00'

    def check_if_snapshot_done(self):
        query = "SELECT event_date FROM marts.dm_malaria_test ORDER BY event_date DESC LIMIT 1"
        result = self.pg_conn.execute_query(query)
        if result:
            return result[0][0]
        else:
            return None

    def fetch_data_in_batches(self):
        snapshot_date = self.check_if_snapshot_done()
        if snapshot_date:
            self.last_processed_value = snapshot_date
            print(">>> Detected Snapshot done:", self.last_processed_value)
        else:
            print(">>> Initial snapshot")

        while True:
            print(f"Fetching batch data where event_date > {self.last_processed_value}")
            query = f"""
            SELECT * FROM report.malaria
            WHERE event_date > '{self.last_processed_value}' 
            ORDER BY event_date ASC LIMIT {self.batch_size}
            """

            
            batch_data, columns = self.mob_conn.execute_query(query)

            if not batch_data:  # No more data
                print("No more data to fetch. Ending batch fetching.")
                break
            
            df = pd.DataFrame(batch_data, columns=columns)
            print("Fetched batch data:", df.shape)

            # Add hash column to the DataFrame
            # df['record_hash'] = df.apply(lambda row: self.hash_record(row), axis=1)

            #df.to_csv("outputwithdps.csv", index=False)
            #pprint()
            # Process and update the last_processed_value
            self.process_data(df)

            # Update the last_processed_value to the latest timestamp in the batch

            self.last_processed_value = df['event_date'].max()
            print("Updated last_processed_value to:", self.last_processed_value)

    def process_data(self, df):
        for _, row in df.iterrows():
            encounter_id = row['encounter_id']
            check_query = "SELECT 1 FROM marts.dm_malaria_test WHERE encounter_id = %s"
            if self.pg_conn.execute_query(check_query, (encounter_id,)):
                # Update existing record
                print(f"Updating record for encounter_id: {encounter_id}")
                update_query = """
                UPDATE marts.dm_malaria
                SET age = %s,
                    sex = %s,
                    complaint= %s,
                    date_of_onset= %s,
                    investigation = %s,
                    malaria_medications= %s,
                    investigation_result= %s,
                    event_date = %s,
                    facility_id_code = %s,
                    dm_date_created = NOW()
                WHERE encounter_id= %s
                """
                self.pg_conn.execute_query(update_query, (
                    row['age'],
                    row['sex'],
                    row['complaints'],
                    row['date_of_onset'],
                    "",
                    row['medication_given'],
                    "",
                    row['event_date'],
                    row['facility_id'],
                    encounter_id
                ))
            else:
                # Insert new record
                print(f"Inserting new record for encounter_id: {encounter_id}")
                insert_query = """
                INSERT INTO marts.dm_malaria_test (
                    age,
                    sex,
                    complaint,
                    date_of_onset,
                    investigation,
                    malaria_medications,
                    investigation_result,
                    dw_date_created,
                    dm_date_created,
                    event_date,
                    encounter_id,
                    facility_id_code,
                    record_hash
                ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s, NOW(),%s, %s,%s,%s)
                """
                self.pg_conn.execute_query(insert_query, (
                  row['age'],
                    row['sex'],
                    row['Complaint'],
                    "",
                    row['investigation'],
                    row['medication'],
                    row['investigation_results'],
                    row['last_updated_date'],
                    row['event_date'],
                    row['facility_id'], encounter_id
                ))
    def start_polling(self):
        """Starts polling to fetch data every 5 minutes."""
        while True:
            print("Starting data fetch cycle.")
            self.fetch_data_in_batches()
            print(f"Waiting for {self.polling_interval} seconds before next fetch.")
            time.sleep(self.polling_interval)
def main():
    # Initialize Mobile connection
    mob_conn = MobileConnection(
        dbname="master",
        user="postgres",
        password="wGMCAE6zFHcyrBmXtus97JPanxvkY4fb",
        host="127.0.0.1",
        port=5434
    )

    # Initialize PostgreSQL connection
    pg_conn = PostgresConnection(
        dbname="HTSDATA",
        user="postgres",
        password="wGMCAE6zFHcyrBmXtus97JPanxvkY4fb",
        host="127.0.0.1",
        port=5431
    )

    if mob_conn.connect() and pg_conn.connect():
        data_fetcher = DataFetcher(mob_conn, pg_conn)
        data_fetcher.start_polling()

    mob_conn.close()
    pg_conn.close()


if __name__ == "__main__":
    main()